<a href="https://colab.research.google.com/github/shreyammb/INNOV8TIGERS/blob/main/predict_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio==1.3.9 numpy pandas scikit-image scikit-learn xgboost lightgbm joblib tqdm shapely

!pip install numpy==1.26.4
import numpy as np
print("NumPy version:", np.__version__)

In [ ]:
import os, math, json
import numpy as np, pandas as pd
import rasterio
from rasterio.warp import reproject, Resampling as WarpResampling
from skimage.feature import local_binary_pattern, canny
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_objects, opening, closing, square
from scipy.ndimage import uniform_filter
from skimage.util import img_as_ubyte
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
import xgboost as xgb, lightgbm as lgb
from joblib import Parallel, delayed
from tqdm import tqdm
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

BASE_TIF = "/content/drive/MyDrive/HYD/S2_BOM_V2.tif"
MASK_TIF = "/content/drive/MyDrive/HYD/sample_mask.tif"

OUT_DIR = "/content/drive/MyDrive/HYD/out_slum"
os.makedirs(OUT_DIR, exist_ok=True)

print("Base:", BASE_TIF)
print("Mask:", MASK_TIF)
print("Out dir:", OUT_DIR)

LBP_RADII = [1, 3]
EDGE_WINDOWS = [3, 7]
ROOF_WINDOWS = [3, 7]
LACUNARITY_WINDOWS = [9, 21]
RANK_GLCM_WINDOW = 7

RF_PARAMS = {"n_estimators":200, "max_depth":12, "class_weight":"balanced", "random_state":42}
XGB_PARAMS = {"n_estimators":300, "max_depth":4, "learning_rate":0.05, "eval_metric":"logloss"}
LGB_PARAMS = {"n_estimators":500, "num_leaves":31, "learning_rate":0.05}

In [ ]:
def read_raster_meta(path):
    with rasterio.open(path) as src:
        names = [src.descriptions[i] or f"Band_{i+1}" for i in range(src.count)]
        meta = src.meta.copy()
    return names, meta

def read_full_raster(path):
    with rasterio.open(path) as src:
        arr = src.read().astype(np.float32)
        names = [src.descriptions[i] or f"Band_{i+1}" for i in range(src.count)]
        meta = src.meta.copy()
    return arr, names, meta

def find_idx_by_sub(substr):
    s=substr.upper()
    for i,nm in enumerate(names):
        if s in nm.upper():
            return i
    return None

In [ ]:
print("Loading raster...")
arr, names, meta = read_full_raster(BASE_TIF)
bands, H, W = arr.shape
print(f"Raster shape: bands={bands}, H={H}, W={W}")
print("Band names (in order):")
for i,nm in enumerate(names,1):
    print(f"{i:02d}. {nm}")

roads_tif = "/content/drive/MyDrive/BOM_WEAKSUP/road_distance_stack.tif"
with rasterio.open(roads_tif) as src:
    roads_arr = src.read(out_shape=(src.count, H, W))
    roads_meta = src.meta

roads_names = [
    "dist_primary",
    "dist_secondary",
    "dist_tertiary",
    "dist_unclassified",
    "dist_residential"
]

arr = np.concatenate([arr, roads_arr], axis=0)
names.extend(roads_names)
bands = arr.shape[0]

print("After adding roads bands:", arr.shape)

band_map = {}
for key in [
    'B2','B3','B4','B8A','B8','B11','B12',
    'CSSI1','CSSI2','NDVI','NDBI','BSI','MNDWI','EVI',
    'GLCM','LBP','B4_ASM','B4_CONTRAST'
]:
    idx = find_idx_by_sub(key)
    if idx is not None:
        band_map[key] = idx
print("Detected band_map (0-based indices):", band_map)

mins = arr.reshape(bands, -1).min(axis=1)
maxs = arr.reshape(bands, -1).max(axis=1)
for i in range(bands):
    print(f"{i+1:02d}. {names[i]}  min={mins[i]:.6g}  max={maxs[i]:.6g}")

const_idx = [i for i in range(bands) if np.isclose(mins[i], maxs[i])]
print("Constant bands (will be dropped):", [names[i] for i in const_idx])
keep_idx = [i for i in range(bands) if i not in const_idx]
arr = arr[keep_idx,:,:]
names = [names[i] for i in keep_idx]
bands = arr.shape[0]
print("Kept bands:", len(names))

ratios = []
for i in range(bands):
    lo, hi = arr[i].min(), arr[i].max()
    if lo == 0:
        r = np.inf if hi != 0 else 1.0
    else:
        r = abs(hi/lo)
    ratios.append(r)

crazy = [i for i,r in enumerate(ratios) if (not np.isfinite(r)) or r > 1e6 or abs(arr[i].max())>1e12]
if crazy:
    print("Bands with extreme ranges detected (will be clipped/logged/dropped):")
    for i in crazy:
        print(f" - {names[i]}  min={arr[i].min():.6g}  max={arr[i].max():.6g}")

for i in crazy:
    band_data = arr[i].ravel()
    p05, p995 = np.nanpercentile(band_data, [0.5, 99.5])
    if not np.isfinite(p05) or not np.isfinite(p995) or p995==p05:
        print("  dropping band (nonfinite percentiles):", names[i])
        arr[i,:,:] = np.nan
    else:
        print(f"  clipping band {names[i]} to [{p05:.3g}, {p995:.3g}]")
        arr[i,:,:] = np.clip(arr[i,:,:], p05, p995)

nan_bands = [i for i in range(arr.shape[0]) if np.all(np.isnan(arr[i]))]
if nan_bands:
    print("Dropping bands with all NaNs:", [names[i] for i in nan_bands])
    keep = [i for i in range(arr.shape[0]) if i not in nan_bands]
    arr = arr[keep,:,:]
    names = [names[i] for i in keep]

bands = arr.shape[0]
print("Final band list used:")
for i,nm in enumerate(names,1):
    print(f"{i:02d}. {nm}")

In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import box

with rasterio.open(MASK_TIF) as ms:
    mask_arr = ms.read(1)
    mask_meta = ms.meta.copy()
    bounds = ms.bounds

with rasterio.open(BASE_TIF) as src:
    window = src.window(*bounds)
    arr_crop_base = src.read(window=window)
    transform_crop = src.window_transform(window)

with rasterio.open(roads_tif) as src:
    arr_crop_roads = src.read(window=window, out_shape=(src.count, arr_crop_base.shape[1], arr_crop_base.shape[2]))

arr_crop = np.concatenate([arr_crop_base, arr_crop_roads], axis=0)


target = np.zeros((arr_crop.shape[1], arr_crop.shape[2]), dtype=np.uint8)
reproject(
    source=mask_arr,
    destination=target,
    src_transform=mask_meta['transform'],
    src_crs=mask_meta['crs'],
    dst_transform=transform_crop,
    dst_crs=src.crs,
    resampling=WarpResampling.nearest
)

mask_aligned = (target != 0).astype(np.uint8)

print("Mask aligned:", mask_aligned.shape, np.unique(mask_aligned))
print("Cropped satellite array:", arr_crop.shape)


ys, xs = np.where(~np.isnan(mask_aligned))
print("Found labeled pixels:", len(ys))


b4_idx = None
for k in ['B4','B8A','B8','B3']:
    idx = find_idx_by_sub(k)
    if idx is not None and idx in keep_idx:
        name_found = [i for i,nm in enumerate(names) if k in nm.upper()]
        if name_found:
            b4_idx = name_found[0]
            break
if b4_idx is None:
    intensity = np.nanmean(arr, axis=0)
else:
    intensity = arr[b4_idx,:,:]
imin, imax = np.nanpercentile(intensity[~np.isnan(intensity)], [1,99])
int_norm = (intensity - imin) / (imax - imin + 1e-9)
int_norm = np.clip(int_norm, 0.0, 1.0)

In [ ]:
# feature extractor - RUN EVERY TIME
def build_feature_stack(tile_arr):
    feats = {}

    for target in ['B2','B3','B4','B8A','B8','B11','B12']:
        idx = find_idx_by_sub(target)
        if idx is not None:
            try:
                i = names.index([n for n in names if target in n.upper()][0])
                feats[target] = tile_arr[i,:,:].astype(np.float32)
            except Exception:
                pass
    for nm in ['CSSI1','CSSI2','NDVI','NDBI','BSI','MNDWI','EVI']:
        idx = find_idx_by_sub(nm)
        if idx is not None:
            match = [n for n in names if nm in n.upper()]
            if match:
                feats[nm] = tile_arr[names.index(match[0]),:,:].astype(np.float32)

    for nm in roads_names:
        if nm in names:
            feats[nm] = tile_arr[names.index(nm), :, :].astype(np.float32)


    for n in names:
        if n.upper().startswith('B4_') or n.upper().startswith('B4 '):
            feats[n] = tile_arr[names.index(n),:,:].astype(np.float32)

    for r in LBP_RADII:
        P = 8*r
        int_norm_safe = np.nan_to_num(int_norm, nan=0.0, posinf=1.0, neginf=0.0)
        int_norm_safe = np.clip(int_norm_safe, 0, 1)
        lbp = local_binary_pattern((int_norm_safe*255).astype(np.uint8), P, r, method='uniform')

        feats[f'LBP_r{r}'] = ((lbp - lbp.min()) / (lbp.max() - lbp.min() + 1e-9)).astype(np.float32)


    edges = canny(int_norm, sigma=1).astype(np.float32)

    for w in EDGE_WINDOWS:
        feats[f'EdgeFrac_{w}'] = uniform_filter(edges, size=w, mode='reflect').astype(np.float32)


    try:
        thr = threshold_otsu(int_norm)
    except:
        thr = (int_norm.mean() + int_norm.std()*0.3)
    roofmask = (int_norm > thr).astype(np.float32)
    for w in ROOF_WINDOWS:
        feats[f'RoofFrac_{w}'] = uniform_filter(roofmask, size=w, mode='reflect').astype(np.float32)


    for w in LACUNARITY_WINDOWS:
        mean = uniform_filter(int_norm, size=w, mode='reflect')
        mean_sq = uniform_filter(int_norm*int_norm, size=w, mode='reflect')
        var = mean_sq - mean*mean
        feats[f'Lacunarity_{w}'] = (var / (mean*mean + 1e-9)).astype(np.float32)

    from skimage.filters import rank
    from skimage.morphology import footprint_rectangle
    selem = footprint_rectangle((RANK_GLCM_WINDOW,RANK_GLCM_WINDOW))
    u8 = img_as_ubyte(int_norm)
    feats['GLCM_entropy'] = rank.entropy(u8, selem).astype(np.float32)
    feats['GLCM_mean'] = rank.mean(u8, selem).astype(np.float32)
    mean = uniform_filter(int_norm.astype(np.float32), size=RANK_GLCM_WINDOW, mode='reflect')
    mean_sq = uniform_filter((int_norm.astype(np.float32)**2), size=RANK_GLCM_WINDOW, mode='reflect')
    feats['GLCM_std'] = np.sqrt(np.maximum(mean_sq - mean*mean, 0)).astype(np.float32)
    feats['GLCM_min'] = rank.minimum(u8, selem).astype(np.float32)
    feats['GLCM_max'] = rank.maximum(u8, selem).astype(np.float32)
    return feats


In [ ]:
# RUN EVERY TIME

print("Building feature table for cropped region where GT exists...")
feats_local = build_feature_stack(arr_crop)
feature_names = list(feats_local.keys())

print("Built feature names:", feature_names)

ys, xs = np.where((mask_aligned==1) | (mask_aligned==0))
rows, labels, coords = [], [], []
for (y,x) in zip(ys,xs):
    fv = [feats_local[f][y,x] for f in feature_names]
    rows.append(fv)
    labels.append(mask_aligned[y,x])
    coords.append((x,y))
X = np.array(rows, dtype=np.float32)
y = np.array(labels, dtype=np.int8)
print("Feature matrix shape:", X.shape, "Labels:", np.bincount(y))


var_per_feat = X.var(axis=0)
const_feats = [feature_names[i] for i,v in enumerate(var_per_feat) if np.isclose(v,0)]
if len(const_feats)>0:
    print("Dropping constant features:", const_feats)
    keep_idx = [i for i in range(len(feature_names)) if feature_names[i] not in const_feats]
    feature_names = [feature_names[i] for i in keep_idx]
    X = X[:, keep_idx]


scaler = RobustScaler()
Xs = scaler.fit_transform(np.nan_to_num(X))


pos_skew_feats = []
for i, nm in enumerate(feature_names):
    col = Xs[:,i]
    if np.nanmin(col) >= 0:
        ratio = (np.nanpercentile(col, 99.9)+1e-12) / (np.nanpercentile(col, 0.1)+1e-12)
        if ratio > 1e3:
            pos_skew_feats.append(nm)
print("Positively skewed features (log transform):", pos_skew_feats)
for nm in pos_skew_feats:
    idx = feature_names.index(nm)
    Xs[:,idx] = np.log1p(Xs[:,idx] - np.min(Xs[:,idx]) + 1e-9)


In [ ]:
cv = StratifiedKFold(n_splits=min(2, max(2,len(y)//10)), shuffle=True, random_state=42)
def cv_eval(model):
    aucs, f1s = [], []
    for tr,va in cv.split(Xs,y):
        model.fit(Xs[tr], y[tr])
        probs = model.predict_proba(Xs[va])[:,1]
        preds = (probs >= 0.5).astype(int)
        aucs.append(roc_auc_score(y[va], probs))
        f1s.append(f1_score(y[va], preds))
    return np.mean(aucs), np.mean(f1s)

rf  = RandomForestClassifier(**RF_PARAMS)
xg  = xgb.XGBClassifier(**XGB_PARAMS, use_label_encoder=False)
lg  = lgb.LGBMClassifier(**LGB_PARAMS)
log = LogisticRegression(class_weight='balanced', solver='liblinear')

models = {'RandomForest':rf, 'XGBoost':xg, 'LightGBM':lg, 'Logistic':log}
results, trained, importances = {}, {}, {}

for name,m in models.items():
    try:
        auc, f1 = cv_eval(m)
        results[name] = {'AUC':auc, 'F1':f1}
        print(f"{name} CV -> AUC: {auc:.4f}, F1: {f1:.4f}")
    except Exception as e:
        print(f"{name} CV failed:", e)

for name,m in models.items():
    print("Training full model on cropped GT region:", name)
    m.fit(Xs, y)
    trained[name] = m
    if hasattr(m, 'feature_importances_'):
        imps = m.feature_importances_
        importances[name] = pd.Series(imps, index=feature_names).sort_values(ascending=False)
    elif hasattr(m, 'coef_'):
        importances[name] = pd.Series(np.abs(m.coef_).ravel(), index=feature_names).sort_values(ascending=False)
    else:
        importances[name] = pd.Series(0, index=feature_names)
    print(" Top features:", importances[name].head(8).to_dict())

imp_df = pd.concat(importances).reset_index()
imp_df.columns = ['model','feature','importance']
agg = imp_df.groupby('feature').importance.mean().sort_values(ascending=False)
print("Aggregated importance (top 20):")
print(agg.head(20))

useless = agg[agg < 1e-3].index.tolist()
print("Useless features candidates (mean importance < 1e-3):", useless)

import joblib

MODELS_DIR = os.path.join(OUT_DIR, "saved_models")
os.makedirs(MODELS_DIR, exist_ok=True)

for name, model in trained.items():
    model_path = os.path.join(MODELS_DIR, f"{name}.joblib")
    joblib.dump(model, model_path)
    print(f"Saved model: {name} -> {model_path}")


summary = {
    "kept_band_names": names,
    "feature_names": feature_names,
    "cv_results": results,
    "agg_importance": agg.head(50).to_dict(),
    "useless_candidates": useless
}
with open(os.path.join(OUT_DIR, "pipeline_summary.json"), "w") as f:
    json.dump(summary, f, indent=2)
print("Saved pipeline summary to", os.path.join(OUT_DIR, "pipeline_summary.json"))



In [ ]:
import rasterio
from rasterio.windows import Window
from rasterio.warp import reproject, Resampling
from sklearn.metrics import precision_score, recall_score, f1_score
import json
import numpy as np
import os
import joblib

tile_size = 1024
stride = 1024

OUT_DIR = "/content/drive/MyDrive/HYD/out_slum/"

full_raster_path = "/content/drive/MyDrive/HYD/S2_BOM_V2.tif"
full_gt_path = "/content/drive/MyDrive/HYD/reprojected_mask_aligned.tif"
roads_tif = "/content/drive/MyDrive/HYD/road_distance_stack.tif"
MODELS_DIR = "/content/drive/MyDrive/HYD/out_slum/saved_models/"


loaded_models = {}
for fname in os.listdir(MODELS_DIR):
    if fname.endswith(".joblib"):
        model_name = os.path.splitext(fname)[0]
        model_path = os.path.join(MODELS_DIR, fname)
        loaded_models[model_name] = joblib.load(model_path)
        print(f"Loaded model: {model_name} from {model_path}")


def process_tile(arr_tile, feature_names, scaler, trained_model):
    feats_tile = build_feature_stack(arr_tile)
    ys, xs = np.where(np.ones((arr_tile.shape[1], arr_tile.shape[2]), dtype=bool))
    rows_tile = []
    for y, x in zip(ys, xs):
        fv = [feats_tile[f][y, x] for f in feature_names]
        rows_tile.append(fv)
    X_tile = np.array(rows_tile, dtype=np.float32)
    X_tile_scaled = scaler.transform(np.nan_to_num(X_tile))
    y_pred = trained_model.predict(X_tile_scaled)
    pred_tile = np.zeros((arr_tile.shape[1], arr_tile.shape[2]), dtype=np.uint8)
    pred_tile[ys, xs] = y_pred
    return pred_tile


evaluation_results = {}

with rasterio.open(full_raster_path) as src_full, rasterio.open(roads_tif) as src_roads:
    H, W = src_full.height, src_full.width
    profile = src_full.profile
    profile.update(dtype=rasterio.uint8, count=1, compress="lzw")

    for name, model in loaded_models.items():
        print(f"Predicting full raster with {name}...")

        out_tif = os.path.join(OUT_DIR, f"pred_{name}.tif")
        with rasterio.open(out_tif, "w", **profile) as dst:

            for i in range(0, H, stride):
                for j in range(0, W, stride):
                    h = min(tile_size, H - i)
                    w = min(tile_size, W - j)

                    arr_tile_base = src_full.read(window=Window(j, i, w, h),
                                                  out_shape=(src_full.count, h, w))
                    arr_tile_roads = src_roads.read(window=Window(j, i, w, h),
                                                    out_shape=(src_roads.count, h, w))
                    arr_tile = np.concatenate([arr_tile_base, arr_tile_roads], axis=0)

                    pred_tile = process_tile(arr_tile, feature_names, scaler, model)

                    dst.write(pred_tile, 1, window=Window(j, i, w, h))

        print(f"Saved predicted TIF for {name} -> {out_tif}")
        with rasterio.open(full_gt_path) as src_gt:
            gt_data = src_gt.read(1)
            gt_meta = src_gt.meta.copy()

        gt_aligned = np.zeros((H, W), dtype=np.uint8)
        reproject(
            source=gt_data,
            destination=gt_aligned,
            src_transform=gt_meta["transform"],
            src_crs=gt_meta["crs"],
            dst_transform=profile["transform"],
            dst_crs=profile["crs"],
            resampling=Resampling.nearest
        )

        with rasterio.open(out_tif) as src_pred:
            pred_full = src_pred.read(1)

        mask = (gt_aligned >= 0)
        y_true = gt_aligned[mask].ravel()
        y_pred_masked = pred_full[mask].ravel()

        precision = precision_score(y_true, y_pred_masked, zero_division=0)
        recall = recall_score(y_true, y_pred_masked, zero_division=0)
        f1 = f1_score(y_true, y_pred_masked, zero_division=0)
        print(f"{name} -> Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

        evaluation_results[name] = {
            "precision": float(precision),
            "recall": float(recall),
            "f1": float(f1)
        }

eval_path = os.path.join(OUT_DIR, "evaluation_full_gt.json")
with open(eval_path, "w") as f:
    json.dump(evaluation_results, f, indent=2)
print("Saved full GT evaluation results to", eval_path)
